# deduplicate ~~bibliographic~~ institution references

The DH-Community is not able to follow citation quides, therefore bibliographic references are quite messy. This script

* extracts all `.//tei:titleStmt//tei:affiliation` elements and writes them to a `.csv` file in the current folder
* this file is feed into `csvdedupe` command line interface which returns `output.csv` with deduplicated files

In [1]:
import glob
import os
import pandas as pd

from lxml import etree as ET

from teipy import TeiReader

In [2]:
try:
    os.makedirs('../indices')
except OSError as e:
    print('../indices alredy exists')

../indices alredy exists


In [3]:
files = glob.glob("../dhd_*/TEI/*.xml")
len(files)

302

In [18]:
def yield_items(files):
    for x in files:
        doc = TeiReader(x)
        doc_id = x
        titel = doc.extract_md()['title']
        counter = 0
        for rs in doc.tree.xpath('.//tei:titleStmt//tei:affiliation', namespaces=doc.ns_tei):
            author_node = rs.getparent()
            author_id = author_node.xpath('./@ref', namespaces=doc.ns_tei)[0]
            rs_text = " ".join("".join(rs.itertext()).split())
            for y in rs_text.split(';'):
                item = {
                    "title": titel,
                    "author_id": author_id,
                    "org": y.strip(),
                    "id": f"{doc_id}__{counter}"
                }
                counter += 1
                yield item

In [23]:
df = pd.DataFrame(yield_items(files))

In [24]:
df.to_csv('orgs.csv')

## run csvdedupe cmd-tool

```shell
csvdedupe orgs.csv --field_names org --output_file org_output.csv
```

* use the result (saved as output.csv) for any further processing
* read output.csv into a `pandas.Dataframe`
* group rows (i.e. bibl entries) by `Cluster ID` (created by dedupe)

In [28]:
deduped = pd.read_csv('org_output.csv')

In [31]:
for i, row in deduped.iterrows():
    doc_id = row['id'].split('.xml__')
#     print(doc_id[0])
    item_id = f"#org__{row['Cluster ID']}"
    node_id = row['author_id']
#     file = f"{doc_id[0]}.xml"
#     try:
#         doc = TeiReader(file)
#     except:
#         continue
#     bibl_node = doc.tree.xpath('.//tei:listBibl//tei:bibl', namespaces=doc.ns_tei)[int(node_id)]
#     bibl_node.set('ref', item_id)
#     doc.tree_to_file(file)

../dhd_2018/TEI/BLUMTRITT_Jonathan_Nutzerunterst_tzung_und_neueste_Entwicklu
../dhd_2018/TEI/BLUMTRITT_Jonathan_Nutzerunterst_tzung_und_neueste_Entwicklu
../dhd_2018/TEI/FRANZINI_Greta_Digital_Medievalist__A_Web_Community_for_Medi
../dhd_2018/TEI/FRANZINI_Greta_Digital_Medievalist__A_Web_Community_for_Medi
../dhd_2018/TEI/FRANZINI_Greta_Digital_Medievalist__A_Web_Community_for_Medi
../dhd_2018/TEI/NASAREK_Robert_Die_Ontologie_historischer_deutschprachiger_B
../dhd_2018/TEI/NASAREK_Robert_Die_Ontologie_historischer_deutschprachiger_B
../dhd_2018/TEI/PFEIFFER_Jasmin_Realit_t_programmieren__Zum_Einfluss_von_Alg
../dhd_2018/TEI/MOELLER_Katrin_Ist_kooperativ_jetzt_umsonst__Die_Ausweisung_
../dhd_2018/TEI/LINDEMANN_David__Lexikographie__Explizite_und_implizite_Vero
../dhd_2018/TEI/LINDEMANN_David__Lexikographie__Explizite_und_implizite_Vero
../dhd_2018/TEI/LINDEMANN_David__Lexikographie__Explizite_und_implizite_Vero
../dhd_2018/TEI/GODLER_Katharina_musilonline___integral_l_sen__Dialogfeld_Di

../dhd_2019/TEI/MANDL_Thomas_Herausforderungen_f_r_die_Klassifikation_histor
../dhd_2019/TEI/SCHOLGER_Martina_Das_Notizbuch_als_Ideenspeicher_und_Forschu
../dhd_2019/TEI/CALVO_TELLO_Jos__Gattungserkennung__ber_500_Jahre
../dhd_2019/TEI/TOSCHKA_Patrick_Semantisch_angereicherte_Pr_sentationsschich
../dhd_2019/TEI/TOSCANO_Roberta_Netzwerkanalyse_f_r_Historiker__Probleme_und
../dhd_2019/TEI/BRUNNER_Annelen_Das_Redewiedergabe_Korpus
../dhd_2019/TEI/BRUNNER_Annelen_Das_Redewiedergabe_Korpus
../dhd_2019/TEI/BRUNNER_Annelen_Das_Redewiedergabe_Korpus
../dhd_2019/TEI/BRUNNER_Annelen_Das_Redewiedergabe_Korpus
../dhd_2019/TEI/BRUNNER_Annelen_Das_Redewiedergabe_Korpus
../dhd_2019/TEI/DIEHR_Franziska_Wie_katalogisiert_man_eigentlich_virtuelle_R
../dhd_2019/TEI/DIEHR_Franziska_Wie_katalogisiert_man_eigentlich_virtuelle_R
../dhd_2019/TEI/R_SSEL_Julia_Auf_alles_gefasst__Metadaten_im_Virtuellen_Kupf
../dhd_2019/TEI/R_SSEL_Julia_Auf_alles_gefasst__Metadaten_im_Virtuellen_Kupf
../dhd_2019/TEI/MAYR_Eva_Vis